# Data Synthesizer

For the BN Model

In [ ]:
import pandas as pd

df = pd.read_pickle('CeneriData/cleaned_2003_2019_platoon.pkl')

ax_cols = []
for i in range(1, 10):
    ax_cols.append('{}_{}'.format('AX_W', i))
    #for j in range(0, 10):
        #ax_cols.append('{}_{}_{}'.format('AX_W', i, j))

for i in range(1, 10):
    ax_cols.append('{}_{}'.format('AX_DIST', i))
    #for j in range(0, 9):
        #ax_cols.append('{}_{}_{}'.format('AX_DIST', i, j))

df_noax = df.drop(columns = ax_cols)

no_col = []
for col in df_noax.columns:
    if col[-1].isdigit() and int(col[-1]) > 5:
        no_col.append(col)

df_sm = df_noax.drop(columns=no_col)

import random

df_sm = df_sm.astype(int)

df_sm.columns

#Comparing platoon lengths of 5
df_5 = df_sm[df_sm.Length == 5]


In [ ]:
#Save df_5 to csv so it can be accessed by the data synthesizer
df_5.to_csv('df_5.csv')

In [1]:
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

import pandas as pd

In [2]:
# input dataset
input_data = 'df_5.csv'
# location of two output files
mode = 'correlated_attribute_mode'
description_file = f'description.json'
synthetic_data = f'sythetic_data.csv'

In [7]:
# An attribute is categorical if its domain size is less than this threshold.
# Here modify the threshold to adapt to the domain size of "education" (which is 14 in input dataset).
threshold_value = 20

# specify categorical attributes
categorical_attributes = {'CLASS_1': True, 'CLASS_2':True,'CLASS_3':True,'CLASS_4':True,'CLASS_5':True, 'Length':True,
                    'Weekday':True,'Hour':True,'AX_1':True,'AX_2':True,'AX_3':True,'AX_4':True,'AX_5':True}

# specify which attributes are candidate keys of input dataset.
#candidate_keys = {'ssn': True}

# A parameter in Differential Privacy. It roughly means that removing a row in the input dataset will not 
# change the probability of getting the same output more than a multiplicative difference of exp(epsilon).
# Increase epsilon value to reduce the injected noises. Set epsilon=0 to turn off differential privacy.
epsilon = 0

# The maximum number of parents in Bayesian network, i.e., the maximum number of incoming edges.
degree_of_bayesian_network = 20

# Number of tuples generated in synthetic dataset.
num_tuples_to_generate = 10000 # Here 32561 is the same as input dataset, but it can be set to another number.

In [ ]:
describer = DataDescriber(category_threshold=threshold_value)
describer.describe_dataset_in_correlated_attribute_mode(dataset_file=input_data, 
                                                        epsilon=epsilon, 
                                                        k=degree_of_bayesian_network,
                                                        attribute_to_is_categorical=categorical_attributes)
describer.save_dataset_description_to_file(description_file)

================ Constructing Bayesian Network (BN) ================
Adding ROOT AX_2
Adding attribute CLASS_2
Adding attribute LENTH_2
Adding attribute GW_TOT_2
Adding attribute Hour
Adding attribute LENTH_1
Adding attribute CLASS_1
Adding attribute GW_TOT_1
Adding attribute LENTH_4
Adding attribute LENTH_5
Adding attribute GW_TOT_3
Adding attribute GW_TOT_5
Adding attribute GW_TOT_4
Adding attribute LENTH_3
Adding attribute CLASS_5
Adding attribute CLASS_3
Adding attribute CLASS_4
Adding attribute Weekday
Adding attribute SPEED_5
Adding attribute AX_1
Adding attribute SPEED_3
Adding attribute AX_5
Adding attribute AX_3
Adding attribute AX_4


In [ ]:
generator = DataGenerator()
generator.generate_dataset_in_correlated_attribute_mode(num_tuples_to_generate, description_file)
generator.save_synthetic_data(synthetic_data)

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
input_df = pd.read_csv(input_data, skipinitialspace=True)
synthetic_df = pd.read_csv(synthetic_data)
# Read attribute description from the dataset description file.
attribute_description = read_json_file(description_file)['attribute_description']

inspector = ModelInspector(input_df, synthetic_df, attribute_description)

In [ ]:

for attribute in synthetic_df.columns:
    inspector.compare_histograms(attribute)

In [ ]:
inspector.mutual_information_heatmap()
